# Neural network trading strategy

We have a 3-D array filled with information about securities of different companies. 

Our goal is to train a neural network on given data and see how it acts in real world.

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import tensorflow as tf

import qnt.data    as qndata
import qnt.stepper as qnstepper
import qnt.stats   as qnstats
import qnt.graph   as qngraph

from scipy.optimize import minimize

ModuleNotFoundError: No module named 'tensorflow'

Downloading our dataset:

In [2]:
data = qndata.load_data(min_date = "2015-01-01",
                        max_date = None,
                        dims     = ("time", "field", "asset"))

fetched chunk 1/11 1s
fetched chunk 2/11 2s
fetched chunk 3/11 3s
fetched chunk 4/11 4s
fetched chunk 5/11 5s
fetched chunk 6/11 6s
fetched chunk 7/11 6s
fetched chunk 8/11 7s
fetched chunk 9/11 8s
fetched chunk 10/11 8s
fetched chunk 11/11 9s
Data loaded 10s


In [3]:
data

<xarray.DataArray 'AMEX:LNG' (time: 1082, field: 9, asset: 2085)>
array([[[32.575,  4.18 , ..., 19.52 , 98.72 ],
        [32.5  ,  4.03 , ..., 18.99 , 98.64 ],
        ...,
        [ 0.5  ,  1.   , ...,  1.   ,  1.   ],
        [ 1.   ,  0.   , ...,  0.   ,  1.   ]],

       [[32.84 ,  4.14 , ..., 19.52 , 98.53 ],
        [32.41 ,  4.08 , ..., 19.45 , 97.1  ],
        ...,
        [ 0.5  ,  1.   , ...,  1.   ,  1.   ],
        [ 1.   ,  0.   , ...,  0.   ,  1.   ]],

       ...,

       [[34.94 ,  3.15 , ...,    nan, 43.25 ],
        [34.038,  3.09 , ...,    nan, 42.97 ],
        ...,
        [ 0.5  ,  1.   , ...,    nan,  1.   ],
        [ 1.   ,  0.   , ...,    nan,  1.   ]],

       [[34.905,  2.86 , ...,    nan, 43.46 ],
        [34.905,  2.85 , ...,    nan, 43.065],
        ...,
        [ 0.5  ,  1.   , ...,    nan,  1.   ],
        [ 1.   ,  0.   , ...,    nan,  1.   ]]])
Coordinates:
  * time     (time) datetime64[ns] 2019-04-22 2019-04-18 ... 2015-01-02
  * field    (field) obj

As we can see, we have 3 coordinates:
 * time (for each day for 3 years)
 * field (information about securities)
 * asset (companies which securities we observing)

# Remove

In [4]:
data = data[:500, :, :120]

In [5]:
data

<xarray.DataArray 'AMEX:LNG' (time: 500, field: 9, asset: 120)>
array([[[32.575,  4.18 , ..., 38.91 , 73.61 ],
        [32.5  ,  4.03 , ..., 38.91 , 72.61 ],
        ...,
        [ 0.5  ,  1.   , ...,  1.   ,  1.   ],
        [ 1.   ,  0.   , ...,  0.   ,  0.   ]],

       [[32.84 ,  4.14 , ..., 38.6  , 73.65 ],
        [32.41 ,  4.08 , ..., 37.79 , 71.92 ],
        ...,
        [ 0.5  ,  1.   , ...,  1.   ,  1.   ],
        [ 1.   ,  0.   , ...,  0.   ,  0.   ]],

       ...,

       [[22.9  ,  4.07 , ..., 15.65 , 46.76 ],
        [22.575,  3.93 , ..., 15.23 , 45.72 ],
        ...,
        [ 0.5  ,  1.   , ...,  1.   ,  1.   ],
        [ 0.   ,  0.   , ...,  0.   ,  0.   ]],

       [[22.96 ,  3.99 , ..., 14.79 , 46.32 ],
        [22.9  ,  3.89 , ..., 14.5  , 45.53 ],
        ...,
        [ 0.5  ,  1.   , ...,  1.   ,  1.   ],
        [ 0.   ,  0.   , ...,  0.   ,  0.   ]]])
Coordinates:
  * time     (time) datetime64[ns] 2019-04-22 2019-04-18 ... 2017-04-26
  * field    (field) objec

# Preparing our data

Our data will consists of 2 things: training data (what we downloaded earlier) and training labels.

## Preparing training data

Downloaded data considers splits but does not considers paid dividents, so we will add 4 fields to our data: "open_bk", "close_bk", "high_bk", "low_bk".

In [6]:
def adjust_by_dividends(data):

    divs_local = data.loc[:, "divs", :].shift({"time": 1}).fillna(0)

    delta = divs_local.cumsum(dim="time")

    bk = data.loc[:, ["open", "close", "high", "low"], :] - delta
    bk = bk.assign_coords(field=["open_bk", "close_bk", "high_bk", "low_bk"])

    return bk

Let's concatenate our data array with new 4 fields: 

In [7]:
data_div = adjust_by_dividends(data)
data_with_divs = xr.concat([data, data_div], dim="field")

In [8]:
data_with_divs

<xarray.DataArray 'AMEX:LNG' (time: 500, field: 13, asset: 120)>
array([[[32.575 ,  4.18  , ..., 38.91  , 73.61  ],
        [32.5   ,  4.03  , ..., 38.91  , 72.61  ],
        ...,
        [33.32  ,  4.18  , ..., 40.12  , 75.91  ],
        [32.5   ,  4.03  , ..., 38.91  , 72.61  ]],

       [[32.84  ,  4.14  , ..., 38.6   , 73.65  ],
        [32.41  ,  4.08  , ..., 37.79  , 71.92  ],
        ...,
        [33.04  ,  4.18  , ..., 39.0848, 75.21  ],
        [32.41  ,  4.08  , ..., 37.79  , 71.92  ]],

       ...,

       [[22.9   ,  4.07  , ..., 15.65  , 46.76  ],
        [22.575 ,  3.93  , ..., 15.23  , 45.72  ],
        ...,
        [23.07  ,  4.07  , ..., 16.45  , 47.25  ],
        [22.575 ,  3.93  , ..., 15.23  , 45.72  ]],

       [[22.96  ,  3.99  , ..., 14.79  , 46.32  ],
        [22.9   ,  3.89  , ..., 14.5   , 45.53  ],
        ...,
        [23.315 ,  4.13  , ..., 16.2   , 47.085 ],
        [22.9   ,  3.89  , ..., 14.5   , 45.53  ]]])
Coordinates:
  * time     (time) datetime64[ns

We should replace 'nan' values with zeros.

In [9]:
data_with_divs = data_with_divs.fillna(0.)

In [10]:
data_with_divs

<xarray.DataArray 'AMEX:LNG' (time: 500, field: 13, asset: 120)>
array([[[32.575 ,  4.18  , ..., 38.91  , 73.61  ],
        [32.5   ,  4.03  , ..., 38.91  , 72.61  ],
        ...,
        [33.32  ,  4.18  , ..., 40.12  , 75.91  ],
        [32.5   ,  4.03  , ..., 38.91  , 72.61  ]],

       [[32.84  ,  4.14  , ..., 38.6   , 73.65  ],
        [32.41  ,  4.08  , ..., 37.79  , 71.92  ],
        ...,
        [33.04  ,  4.18  , ..., 39.0848, 75.21  ],
        [32.41  ,  4.08  , ..., 37.79  , 71.92  ]],

       ...,

       [[22.9   ,  4.07  , ..., 15.65  , 46.76  ],
        [22.575 ,  3.93  , ..., 15.23  , 45.72  ],
        ...,
        [23.07  ,  4.07  , ..., 16.45  , 47.25  ],
        [22.575 ,  3.93  , ..., 15.23  , 45.72  ]],

       [[22.96  ,  3.99  , ..., 14.79  , 46.32  ],
        [22.9   ,  3.89  , ..., 14.5   , 45.53  ],
        ...,
        [23.315 ,  4.13  , ..., 16.2   , 47.085 ],
        [22.9   ,  3.89  , ..., 14.5   , 45.53  ]]])
Coordinates:
  * time     (time) datetime64[ns

## Preparing training labels 

We will use an optimizer to calculate optimal weights (2085 parameters from 0 to 1 which in sum give 1) which leads to most profit for a given period.

In [11]:
def getBestStrategyOn2Day(firstDay, secondDay):
    
    def f(moneyDistr):
        result = 0.
        for i in range(len(firstDay)):
            if firstDay[i]==0:
                result += 0
            else:
                result += (secondDay[i] / firstDay[i])*moneyDistr[i]
        return result
    
    def rosen(x):
        x = x / x.sum()
        return 1/f(x)
    
    def mtrx(x):
        lst = secondDay / firstDay
        return lst
        

    result = np.zeros((len(firstDay)))
        
    bnd = tuple(np.ones(len(firstDay)))    
    bnd2 = tuple(np.ones(len(firstDay))*100)    
    bnd3 = list(zip(bnd,bnd2))
                      
    res = minimize(rosen, result, method='TNC',jac = mtrx, bounds = bnd3, options = \
                   {'maxiter' : 50, 'eta' : 0.15, 'stepmx' : 5.0,\
                    'accuracy' : 10.0e-3,'ftol' : 10.0e-3, 'gtol' : 1.0e-3, 'rescale' : 0.5})
    
    return (res.x / (res.x.sum()))

In [12]:
# Counting prices
prices = data_with_divs.loc[::-1, "close", :]

# Creating weights as same as prices
weights = prices

for k in range(len(prices)-1):
    print(k)
    weights[k] = getBestStrategyOn2Day(prices[k, : ],prices[k + 1, : ])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
# Making training data as same as data with divs
training_data = data_with_divs[::-1]

# Constants
num = len(training_data)
ass = len(training_data.asset)
fiel = len(training_data.field)

# Making training data to array
training_data = training_data.values

# Reshaping training data
training_data = training_data.reshape(num, ass, fiel, 1)

In [14]:
# Making training labels as same as weights
training_labels = weights

#Making training labels vectors
#raining_labels = training_labels.values

In [15]:
weights

<xarray.DataArray 'AMEX:LNG' (time: 500, asset: 120)>
array([[8.333333e-03, 8.333333e-03, 8.333333e-03, ..., 8.333333e-03,
        8.333333e-03, 8.333333e-03],
       [8.333333e-03, 8.333333e-03, 8.333333e-03, ..., 8.333333e-03,
        8.333333e-03, 8.333333e-03],
       [8.333333e-03, 8.333333e-03, 8.333333e-03, ..., 8.333333e-03,
        8.333333e-03, 8.333333e-03],
       ...,
       [8.333333e-03, 8.333333e-03, 8.333333e-03, ..., 8.333333e-03,
        8.333333e-03, 8.333333e-03],
       [8.333333e-03, 8.333333e-03, 8.333333e-03, ..., 8.333333e-03,
        8.333333e-03, 8.333333e-03],
       [3.326000e+01, 4.050000e+00, 8.542000e-01, ..., 3.942000e+01,
        3.952000e+01, 7.543000e+01]])
Coordinates:
  * time     (time) datetime64[ns] 2017-04-26 2017-04-27 ... 2019-04-22
  * asset    (asset) object 'AMEX:LNG' 'AMEX:NG' ... 'NASDAQ:BOLD' 'NASDAQ:BPMC'
    field    <U5 'close'

In [16]:
training_data

array([[[[2.29600000e+01],
         [3.99000000e+00],
         [3.00000000e+00],
         ...,
         [4.04516000e+03],
         [2.89440000e+02],
         [2.60080000e+02]],

        [[3.87000000e+01],
         [3.47700000e+01],
         [2.91000000e+02],
         ...,
         [1.06800000e+03],
         [1.58820000e+02],
         [1.63500000e+01]],

        [[2.09020000e+02],
         [9.60000000e+00],
         [3.61240000e+02],
         ...,
         [1.95700000e+01],
         [6.18100000e+01],
         [2.29900000e+01]],

        ...,

        [[8.59000000e+00],
         [1.52000000e-01],
         [5.74340263e+02],
         ...,
         [5.50800000e-01],
         [0.00000000e+00],
         [8.59400000e+01]],

        [[0.00000000e+00],
         [2.28700000e+01],
         [1.78500000e+01],
         ...,
         [0.00000000e+00],
         [3.90000000e+02],
         [1.84900000e+03]],

        [[1.67742000e+03],
         [0.00000000e+00],
         [2.98926440e+02],
         ...,
 

In [17]:
len(training_data)

500

In [18]:
training_labels

<xarray.DataArray 'AMEX:LNG' (time: 500, asset: 120)>
array([[8.333333e-03, 8.333333e-03, 8.333333e-03, ..., 8.333333e-03,
        8.333333e-03, 8.333333e-03],
       [8.333333e-03, 8.333333e-03, 8.333333e-03, ..., 8.333333e-03,
        8.333333e-03, 8.333333e-03],
       [8.333333e-03, 8.333333e-03, 8.333333e-03, ..., 8.333333e-03,
        8.333333e-03, 8.333333e-03],
       ...,
       [8.333333e-03, 8.333333e-03, 8.333333e-03, ..., 8.333333e-03,
        8.333333e-03, 8.333333e-03],
       [8.333333e-03, 8.333333e-03, 8.333333e-03, ..., 8.333333e-03,
        8.333333e-03, 8.333333e-03],
       [3.326000e+01, 4.050000e+00, 8.542000e-01, ..., 3.942000e+01,
        3.952000e+01, 7.543000e+01]])
Coordinates:
  * time     (time) datetime64[ns] 2017-04-26 2017-04-27 ... 2019-04-22
  * asset    (asset) object 'AMEX:LNG' 'AMEX:NG' ... 'NASDAQ:BOLD' 'NASDAQ:BPMC'
    field    <U5 'close'

In [19]:
len(training_labels)

500

# Building neural network model 

In [20]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(32, (3,3), activation=tf.nn.relu, input_shape = (ass, fiel, 1)),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.BatchNormalization(),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),                                    
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),                                    
                                    tf.keras.layers.Dense(ass, activation=tf.nn.relu)])

Instructions for updating:
Colocations handled automatically by placer.


In [21]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer='adam',
              loss=tf.keras.losses.mean_squared_logarithmic_error,
              metrics = ['accuracy'])

In [22]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = {}):
        if (logs.get('acc') > 0.81):
            print("\nReached 81.0% accuracy so cancelling training!")
            self.model.stop_training = True
myCallback = MyCallback()

## Run our model

In [23]:
model.fit(training_data, training_labels, epochs=25, callbacks=[myCallback])

Instructions for updating:
Use tf.cast instead.
Epoch 1/25
500/500 [==============================] - 6s 13ms/sample - loss: 0.0792 - acc: 0.6180
Epoch 2/25
480/500 [===========================>..] - ETA: 0s - loss: 0.0340 - acc: 0.9979
Reached 81.0% accuracy so cancelling training!
500/500 [==============================] - 5s 10ms/sample - loss: 0.0327 - acc: 0.9960


In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 118, 11, 32)       320       
_________________________________________________________________
flatten (Flatten)            (None, 41536)             0         
_________________________________________________________________
batch_normalization_v1 (Batc (None, 41536)             166144    
_________________________________________________________________
dense (Dense)                (None, 128)               5316736   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               66048     
_________________________________________________________________
dense_2 (Dense)              (None, 120)               61560     
Total params: 5,610,808
Trainable params: 5,527,736
Non-trainable params: 83,072
_____________________________________________________________

# What we got

Calculating statistics depending on our weights:

In [25]:
def calc_risk(data, period=3):

    inputs = data.loc[::-1, ["close", "close_bk", "high_bk", "low_bk"], :]

    # !! data.loc[::-1,
    inputs = inputs.assign_coords(field=["close_forward", "close_bk_forward", "high_bk_forward", "low_bk_forward"])

    d1 = inputs.loc[:, "high_bk_forward", :] - inputs.loc[:, "low_bk_forward", :] # диапазон цены сегодня
    d2 = abs(inputs.loc[:, "close_bk_forward", :] - inputs.loc[:, "close_bk_forward", :].shift({"time":1})).drop("field") # диапазон цены за сутк
    d3 = inputs.loc[:, "close_forward", :].shift({"time":1}).drop("field")

    # вычисление максимума
    dd = xr.concat([d1, d2], dim="new").max(dim="new")

    # относительная доходность
    dv = dd / d3

    # вычисление среднего за последние 20 дней
    risk = dv.rolling({"time":period}, min_periods=period).mean(skipna=True).ffill("time")
    risk.coords["field"] = "risk"

    # !! data.loc[::-1,
    return risk.loc[::-1, :]

In [26]:
def step(data):

        # Creating 4 column dividents
    data_div_r = adjust_by_dividends(data)

    # Connecting data with divident columns
    cleared_data = xr.concat([data, data_div_r], dim="field")

    # Clearing data from 'nan'
    cleared_data = cleared_data.fillna(0.)

    # Calculating risks
    risk = calc_risk(cleared_data)

    # Constants
    num = len(cleared_data)
    ass = len(cleared_data.asset)
    fiel = len(cleared_data.field)

    # Making training data to array
    cleared_data = cleared_data.values

    # Reshaping it (adding 1 dimension)
    cleared_data = cleared_data.reshape( num, ass, fiel, 1)

    # Predicting weights
    weights = model.predict(cleared_data)

    # Adding axes
    weights_with_axes = xr.DataArray(
        weights,
        dims   = ['time', 'asset'],
        coords = {'time':data.time, 'asset':data.asset})


    risk = risk.fillna(1)
   
    
    weights_with_axes = weights_with_axes / risk

    weights_sum = weights_with_axes.sum("asset")
    
    weights_norm = weights_with_axes / weights_sum # полное инвестирование
    
    weights_with_axes = weights_norm.fillna(0.001)
    
    weights_with_axes = weights_with_axes / weights_with_axes.sum()
    
    return weights_with_axes[-1]

In [27]:
data = qndata.load_data(min_date = "2015-01-01",
                        max_date = None,
                        dims     = ("time", "field", "asset"))

fetched chunk 1/11 1s
fetched chunk 2/11 3s
fetched chunk 3/11 4s
fetched chunk 4/11 5s
fetched chunk 5/11 5s
fetched chunk 6/11 6s
fetched chunk 7/11 7s
fetched chunk 8/11 8s
fetched chunk 9/11 8s
fetched chunk 10/11 9s
fetched chunk 11/11 10s
Data loaded 10s


In [28]:
data = data [500:1000, :, :120]

In [29]:
init_data_length = 2

output = qnstepper.test_strategy(data, step=step, init_data_length=init_data_length)

Testing started...
Testing progress: 40/498 5s
Testing progress: 64/498 10s
Testing progress: 81/498 15s
Testing progress: 97/498 20s
Testing progress: 112/498 26s
Testing progress: 125/498 31s
Testing progress: 137/498 36s
Testing progress: 148/498 42s
Testing progress: 158/498 47s
Testing progress: 167/498 52s
Testing progress: 176/498 57s
Testing progress: 185/498 63s
Testing progress: 194/498 68s
Testing progress: 202/498 74s
Testing progress: 210/498 79s
Testing progress: 218/498 85s
Testing progress: 225/498 90s
Testing progress: 232/498 95s
Testing progress: 238/498 101s
Testing progress: 244/498 107s
Testing progress: 250/498 113s
Testing progress: 255/498 118s
Testing progress: 261/498 124s
Testing progress: 267/498 129s
Testing progress: 273/498 136s
Testing progress: 278/498 141s
Testing progress: 283/498 147s
Testing progress: 288/498 152s
Testing progress: 293/498 157s
Testing progress: 298/498 163s
Testing progress: 303/498 169s
Testing progress: 308/498 174s
Testing prog

In [30]:
output

<xarray.DataArray (time: 498, asset: 120)>
array([[2.777778e-03, 2.777778e-03, 2.777778e-03, ..., 2.777778e-03,
        2.777778e-03, 2.777778e-03],
       [7.352941e-04, 7.352941e-04, 7.352941e-04, ..., 7.352941e-04,
        7.352941e-04, 7.352941e-04],
       [6.756757e-04, 6.756757e-04, 6.756757e-04, ..., 6.756757e-04,
        6.756757e-04, 6.756757e-04],
       ...,
       [1.477541e-05, 1.477541e-05, 1.477541e-05, ..., 1.477541e-05,
        1.477541e-05, 1.477541e-05],
       [1.474926e-05, 1.474926e-05, 1.474926e-05, ..., 1.474926e-05,
        1.474926e-05, 1.474926e-05],
       [1.472320e-05, 1.472320e-05, 1.472320e-05, ..., 1.472320e-05,
        1.472320e-05, 1.472320e-05]])
Coordinates:
  * time     (time) datetime64[ns] 2015-05-05 2015-05-06 ... 2017-04-25
  * asset    (asset) object 'AMEX:LNG' 'AMEX:NG' ... 'NASDAQ:BOLD' 'NASDAQ:BPMC'

In [31]:
stat = qnstats.calc_stat(data, output, slippage_factor=0.05)
display(stat.to_pandas().tail())

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover
time,,,,,,,,,,
2017-04-19,1.000982,0.000002,0.001329,-0.000439,0.001551,0.385191,0.000512,1.0,120.0,0.000209
2017-04-20,1.000994,0.000012,0.001328,-0.000426,0.001551,0.389666,0.000517,1.0,120.0,0.000209
2017-04-21,1.000986,-0.000008,0.001327,-0.000434,0.001551,0.386050,0.000512,1.0,120.0,0.000208
2017-04-24,1.001002,0.000016,0.001325,-0.000418,0.001551,0.391939,0.000519,1.0,120.0,0.000208
2017-04-25,1.001018,0.000016,0.001324,-0.000403,0.001551,0.397627,0.000526,1.0,120.0,0.000208


Define method to print statistics:

In [32]:
def print_stat(stat):

    days    = len(stat.coords["time"])
    returns = stat.loc[:, "relative_return"]
    equity  = stat.loc[:, "equity"]

    sharpe_ratio = qnstats.calc_sharpe_ratio_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1]

    profit = qnstats.calc_mean_return_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1]*100.0

    volatility = qnstats.calc_volatility_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1]*100.0

    max_ddown = qnstats.calc_max_drawdown(qnstats.calc_underwater(equity)).to_pandas().values[-1]*100.0

    print("Sharpe Ratio         : ", "{0:.3f}".format(sharpe_ratio))
    print("Mean Return [%]      : ", "{0:.3f}".format(profit))
    print("Volatility  [%]      : ", "{0:.3f}".format(volatility))
    print("Maximum Drawdown [%] : ", "{0:.3f}".format(-max_ddown))

In [33]:
print_stat(stat)

Sharpe Ratio         :  0.398
Mean Return [%]      :  0.053
Volatility  [%]      :  0.132
Maximum Drawdown [%] :  -0.155


Writing output:

In [34]:
qnstepper.write_output(output)

write output: /home/username/fractions.nc.gz
